In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import h2o
from h2o.automl import H2OAutoML
h2o.init(nthreads=7, 
         max_mem_size='6G')

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,38 mins 52 secs
H2O cluster timezone:,Europe/Paris
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.3
H2O cluster version age:,25 days
H2O cluster name:,H2O_from_python_edu_yuaa4k
H2O cluster total nodes:,1
H2O cluster free memory:,1.236 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [8]:
names = ['trainLow', 'trainMid', 'trainHigh', 'testLow', 'testMid', 'testHigh']
trainLow, trainMid, trainHigh, testLow, testMid, testHigh = [h2o.import_file(f'../cleaned-data/09-{e}allCatAsDummies-split3Way.csv') for e in names]
                                                                            

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [9]:
trainLow

id,carat,depth,table,x,y,z,cut_0,cut_1,cut_2,cut_3,cut_4,color_1,color_2,color_3,color_4,color_5,color_6,color_7,clarity_0,clarity_1,clarity_2,clarity_3,clarity_4,clarity_5,clarity_6,clarity_7,price
0,0.78,61.5,58,5.93,5.98,3.66,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,3446
1,0.31,60.8,56,4.37,4.32,2.64,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,732
2,0.3,62.3,54,4.3,4.34,2.69,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,475
3,1.04,62,58,6.54,6.46,4.03,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,9552
4,0.65,61.4,55,5.58,5.62,3.44,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1276
5,0.9,62.2,60,6.15,6.19,3.84,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,3662
6,0.31,62.3,55,4.34,4.36,2.71,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,446
7,0.71,64.6,59,5.62,5.59,3.62,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,2902
9,1.1,61.3,58,6.64,6.71,4.09,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,4791
10,1.19,59.9,61,6.85,6.91,4.12,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,5644


In [11]:
xCols = trainLow.col_names[1:-1]
xCols

['carat',
 'depth',
 'table',
 'x',
 'y',
 'z',
 'cut_0',
 'cut_1',
 'cut_2',
 'cut_3',
 'cut_4',
 'color_1',
 'color_2',
 'color_3',
 'color_4',
 'color_5',
 'color_6',
 'color_7',
 'clarity_0',
 'clarity_1',
 'clarity_2',
 'clarity_3',
 'clarity_4',
 'clarity_5',
 'clarity_6',
 'clarity_7']

In [21]:
def trainAutoMLModel(name, train, test, runtime):
    
    aml = H2OAutoML(max_runtime_secs = runtime, project_name = name, keep_cross_validation_models=True)
    aml.train(x=xCols, y = 'price', training_frame = train)
    pred = aml.leader.predict(test)
    display(pred)
    display(aml.leaderboard)
    #combTable = test['id'].cbind(pred)
    combTable = combTable.rename(columns={'predict':'price'})
    h2o.export_file(frame=combTable, path=f'../predictions/raw/{name}.csv', force=True)

In [20]:
trainAutoMLModel('trainLow',trainLow, testLow, 5)

AutoML progress: |
10:49:55.105: New models will be added to existing leaderboard trainLow@@price (leaderboard frame=null) with already 3 models.
10:50:01.131: StackedEnsemble_AllModels_AutoML_20200302_104955 [StackedEnsemble all (built using all AutoML models)] failed: water.exceptions.H2OIllegalArgumentException: Failed to find the xval predictions frame. . .  Looks like keep_cross_validation_predictions wasn't set when building the models, or the frame was deleted.
10:50:24.281: New models will be added to existing leaderboard trainLow@@price (leaderboard frame=null) with already 4 models.
10:50:31.295: StackedEnsemble_AllModels_AutoML_20200302_105024 [StackedEnsemble all (built using all AutoML models)] failed: water.exceptions.H2OIllegalArgumentException: Failed to find the xval predictions frame. . .  Looks like keep_cross_validation_predictions wasn't set when building the models, or the frame was deleted.
10:50:42.92: New models will be added to existing leaderboard trainLow@@p

predict
4571.11
1038.42
6835.44
1409.24
2338.97
1882.12
1885.99
3515.84
13050.5
3485.1


model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20200302_104837,263208,513.038,263208,276.044,0.113163
XGBoost_1_AutoML_20200302_104837,761735,872.774,761735,522.456,0.175788
XGBoost_2_AutoML_20200302_104837,6.60816e+06,2570.63,6.60816e+06,1798.28,0.723486
XGBoost_1_AutoML_20200302_104955,7.24436e+06,2691.53,7.24436e+06,1878.5,0.779592
XGBoost_1_AutoML_20200302_105042,8.49434e+06,2914.51,8.49434e+06,2048.16,0.879597
XGBoost_1_AutoML_20200302_105024,9.73889e+06,3120.72,9.73889e+06,2202.43,1.00021


Export File progress: |███████████████████████████████████████████████████| 100%


In [22]:
frames = trainLow, trainMid, trainHigh, testLow, testMid, testHigh
trainAutoMLModel('lowCarat', trainLow, testLow, 180)
trainAutoMLModel('midCarat', trainMid, testMid, 180)
trainAutoMLModel('highCarat', trainHigh, testHigh, 180)

AutoML progress: |████████████████████████████████████████████████████████| 100%
xgboost prediction progress: |████████████████████████████████████████████| 100%


predict
4430.67
1040.97
7004.1
1473.78
2240.11
1971.93
1798.88
3492.3
15347.5
3692.23


model_id,mean_residual_deviance,rmse,mse,mae,rmsle
XGBoost_1_AutoML_20200302_105331,186175,431.48,186175,230.215,0.0932024
StackedEnsemble_BestOfFamily_AutoML_20200302_105331,186491,431.846,186491,229.682,0.0924055
StackedEnsemble_AllModels_AutoML_20200302_105331,186533,431.895,186533,228.363,0.0913529
XGBoost_2_AutoML_20200302_105331,240632,490.543,240632,259.011,0.0956993
DRF_1_AutoML_20200302_105331,283504,532.451,283504,281.933,0.112541
XGBoost_3_AutoML_20200302_105331,537433,733.098,537433,396.098,0.158313
GBM_1_AutoML_20200302_105331,6.15328e+06,2480.58,6.15328e+06,1868.94,0.854624
GLM_1_AutoML_20200302_105331,1.11047e+07,3332.36,1.11047e+07,2572.5,1.05989


Export File progress: |███████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%


predict
16911
16172.1
15723.8
16707.5
15623.6
13096.7
16787.1
16711.2
15093.4
16057.4


model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20200302_105634,2.74256e+06,1656.07,2.74256e+06,1291.61,0.117709
StackedEnsemble_BestOfFamily_AutoML_20200302_105634,2.74429e+06,1656.59,2.74429e+06,1298.62,0.118052
XGBoost_3_AutoML_20200302_105634,2.7679e+06,1663.7,2.7679e+06,1320.01,0.1185
XGBoost_1_AutoML_20200302_105634,2.76798e+06,1663.72,2.76798e+06,1308.5,0.118093
XGBoost_2_AutoML_20200302_105634,2.78359e+06,1668.41,2.78359e+06,1319.5,0.118371
GBM_2_AutoML_20200302_105634,2.79551e+06,1671.98,2.79551e+06,1311.88,0.119486
GBM_3_AutoML_20200302_105634,2.79883e+06,1672.97,2.79883e+06,1315.61,0.119464
XGBoost_grid__1_AutoML_20200302_105634_model_1,2.8046e+06,1674.69,2.8046e+06,1332.83,0.11939
XGBoost_grid__1_AutoML_20200302_105634_model_2,2.81191e+06,1676.87,2.81191e+06,1327.72,0.119621
GBM_grid__1_AutoML_20200302_105634_model_8,2.82373e+06,1680.39,2.82373e+06,1340.03,0.119339


Export File progress: |███████████████████████████████████████████████████| 100%
AutoML progress: |██████████
10:59:52.601: Skipping training of model GBM_5_AutoML_20200302_105939 due to exception: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for GBM model: GBM_5_AutoML_20200302_105939.  Details: ERRR on field: _min_rows: The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 47.0.


███████████████████████████████████████Failed polling AutoML progress log: HTTP 500 Server Error:
Server error java.lang.ArrayIndexOutOfBoundsException:
  Error: Index 141 out of bounds for length 141
  Request: None

███████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%


predict
15700.7
17741.4
17758
17250.3
11071.6
14052
15420.4
17095.8
9405.55
16464.8


model_id,mean_residual_deviance,rmse,mse,mae,rmsle
GBM_grid__1_AutoML_20200302_105939_model_63,4.90189e+06,2214.02,4.90189e+06,1764.75,0.175838
GBM_grid__1_AutoML_20200302_105939_model_134,4.94327e+06,2223.35,4.94327e+06,1804.74,0.168976
XGBoost_1_AutoML_20200302_105939,5.16039e+06,2271.65,5.16039e+06,1899.05,0.180658
GBM_grid__1_AutoML_20200302_105939_model_30,5.17042e+06,2273.86,5.17042e+06,1891.53,0.183034
XGBoost_grid__1_AutoML_20200302_105939_model_25,5.27894e+06,2297.6,5.27894e+06,1896.78,0.181148
XGBoost_grid__1_AutoML_20200302_105939_model_26,5.3288e+06,2308.42,5.3288e+06,1881.12,0.178022
GBM_grid__1_AutoML_20200302_105939_model_175,5.45147e+06,2334.84,5.45147e+06,1896.46,0.189997
XGBoost_grid__1_AutoML_20200302_105939_model_10,5.49379e+06,2343.88,5.49379e+06,1950.73,0.184316
GBM_grid__1_AutoML_20200302_105939_model_131,5.52228e+06,2349.95,5.52228e+06,1942.26,0.194471
XRT_1_AutoML_20200302_105939,5.52259e+06,2350.02,5.52259e+06,1914.68,0.183479


Export File progress: |███████████████████████████████████████████████████| 100%
